In [1]:
import os, javalang
from pprint import pprint
                
def parse_java_files_file_level_interface_class(PATH, root_PATH):
    with open("{}_count_interface_class.csv".format(root_PATH), "w") as ff:  ## file for field
        ff.write("{},{},{},{},{}\n".format("file", "num_of_anony", "num_of_interface", "num_of_class", "line_of_code"))
        ## start to iterate the files:
        for file in os.listdir(PATH):
            if ".java" not in file:
                continue
            #####
            ## within each code file:
            #####
            try:
                with open(os.path.join(PATH, file), "r") as pf:
                    content = pf.readlines()
                    code_string = "".join(content)
                    tree = javalang.parse.parse(code_string)
                    #####
                    counter_of_anony = "??"
                    #####
                    counter_interface = 0
                    ## https://github.com/c2nes/javalang/blob/master/javalang/tree.py
                    for path, node in tree.filter(javalang.tree.InterfaceDeclaration):
                        counter_interface += 1
                    #####
                    counter_class = 0
                    ## https://github.com/c2nes/javalang/blob/master/javalang/tree.py
                    for path, node in tree.filter(javalang.tree.ClassDeclaration):
                        counter_class += 1
                    ff.write("{},{},{},{},{}\n".format(file, counter_of_anony, counter_interface, counter_class,
                                                       len(content)))
            except:
                print("failed:", file)


def parse_java_files_class_level_method(PATH, root_PATH):
    with open("{}_method.csv".format(root_PATH), "w") as ff:  ## file for field
        ff.write(
            "{},{},{},{},{},{},{}\n".format(
                "file",
                "NOM_avg", "NOM_max", "NOM_total",
                "NSM_avg", "NSM_max", "NSM_total"
            )
        )
        ## start to iterate the files:
        for file in os.listdir(PATH):
            if ".java" not in file:
                continue
            #####
            ## within each code file:
            #####
            try:
                with open(os.path.join(PATH, file), "r") as pf:
                    content = pf.readlines()
                code_string = "".join(content)
                tree = javalang.parse.parse(code_string)
                normal = {
                    "max": 0,
                    "avg": 0,
                    "num_of_class": 0,
                    "total": 0
                }
                static = {
                    "max": 0,
                    "avg": 0,
                    "num_of_class": 0,
                    "total": 0
                }
                ## https://github.com/c2nes/javalang/blob/master/javalang/tree.py
                for path, node in tree.filter(javalang.tree.ClassDeclaration):
                    #####
                    ## within each class:
                    #####
                    ## methods:
                    normal["num_of_class"] += 1
                    static["num_of_class"] += 1
                    counter_method = 0
                    counter_method_static = 0
                    for path1, node1 in node.filter(
                            javalang.tree.MethodDeclaration):  ## node1 is each line of MethodDeclaration
                        if "static" in node1.modifiers:
                            counter_method_static += 1
                        else:
                            counter_method += 1
                    normal["max"] = normal["max"] if counter_method <= normal["max"] else counter_method
                    static["max"] = static["max"] if counter_method_static <= static["max"] else counter_method_static
                    normal["total"] += counter_method
                    static["total"] += counter_method_static
                normal["avg"] = normal["total"] / normal["num_of_class"] if normal["num_of_class"] > 0 else 1  ### IF THE normal["num_of_class"] IS 0, THEN WE WILL MANUALLY CHANGE IT TO 1
                static["avg"] = static["total"] / static["num_of_class"] if normal["num_of_class"] > 0 else 1  ### IF THE normal["num_of_class"] IS 0, THEN WE WILL MANUALLY CHANGE IT TO 1
                ff.write(
                    "{},{},{},{},{},{},{}\n".format(
                        file,
                        normal["avg"], normal["max"], normal["total"],
                        static["avg"], static["max"], static["total"],
                    )
                )
            except:
                print("failed:", file)
                continue


def parse_java_files_class_level_field(PATH, root_PATH):
    with open("{}_field.csv".format(root_PATH), "w") as ff:  ## file for field
        ff.write(
            "{},{},{},{},{},{},{}\n".format(
                "file",
                "NOF_avg", "NOF_max", "NOF_total",
                "NSF_avg", "NSF_max", "NSF_total"
            )
        )
        ## start to iterate the files:
        for file in os.listdir(PATH):
            if ".java" not in file:
                continue
            #####
            ## within each code file:
            #####
            try:
                with open(os.path.join(PATH, file), "r") as pf:
                    content = pf.readlines()
                code_string = "".join(content)
                tree = javalang.parse.parse(code_string)
                normal = {
                    "max": 0,
                    "avg": 0,
                    "num_of_class": 0,
                    "total": 0
                }
                static = {
                    "max": 0,
                    "avg": 0,
                    "num_of_class": 0,
                    "total": 0
                }
                ## https://github.com/c2nes/javalang/blob/master/javalang/tree.py
                for path, node in tree.filter(javalang.tree.ClassDeclaration):
                    #####
                    ## within each class:
                    #####
                    # print(node.name)
                    ## fields:
                    normal["num_of_class"] += 1
                    static["num_of_class"] += 1
                    counter_field = 0
                    counter_field_static = 0
                    for path1, node1 in node.filter(
                            javalang.tree.FieldDeclaration):  ## node1 is each line of FieldDeclaration
                        if "static" in node1.modifiers:
                            for variable_declarator in node1.declarators:  ## basically one declarator is just one line. Within 1 line, there can be several varaibles.
                                # print("static: ", node1.modifiers, variable_declarator.name)
                                counter_field_static += 1
                        else:
                            for variable_declarator in node1.declarators:  ## basically one declarator is just one line. Within 1 line, there can be several varaibles.
                                # print("not: ", node1.modifiers, variable_declarator.name)
                                counter_field += 1
                    normal["max"] = normal["max"] if counter_field <= normal["max"] else counter_field
                    static["max"] = static["max"] if counter_field_static <= static["max"] else counter_field_static
                    normal["total"] += counter_field
                    static["total"] += counter_field_static
                normal["avg"] = normal["total"] / normal["num_of_class"] if normal["num_of_class"] > 0 else 1  ### IF THE normal["num_of_class"] IS 0, THEN WE WILL MANUALLY CHANGE IT TO 1
                static["avg"] = static["total"] / static["num_of_class"] if normal["num_of_class"] > 0 else 1  ### IF THE normal["num_of_class"] IS 0, THEN WE WILL MANUALLY CHANGE IT TO 1
                ff.write(
                    "{},{},{},{},{},{},{}\n".format(
                        file,
                        normal["avg"], normal["max"], normal["total"],
                        static["avg"], static["max"], static["total"],
                    )
                )
            except:
                print("failed:", file)

### platform

In [2]:
parse_java_files_class_level_field("Code/org.eclipse.platform_renamed", "org.eclipse.platform")
parse_java_files_class_level_method("Code/org.eclipse.platform_renamed", "org.eclipse.platform")

parse_java_files_file_level_interface_class("Code/org.eclipse.platform_renamed", "org.eclipse.platform")

### jdt.core

In [ ]:
parse_java_files_class_level_field("Code/org.eclipse.jdt.core_renamed", "org.eclipse.jdt.core")
parse_java_files_class_level_method("Code/org.eclipse.jdt.core_renamed", "org.eclipse.jdt.core")

parse_java_files_file_level_interface_class("Code/org.eclipse.jdt.core_renamed", "org.eclipse.jdt.core")

In [ ]:
# for file in os.listdir(PATH):
#         if ".java" not in file:
#             continue
#         #####
#         ## within each code file: 
#         #####
#         with open(os.path.join(PATH, file), "r") as pf:
#             content = pf.readlines()
#             code_string = "".join(content)
#             tree = javalang.parse.parse(code_string)
#             ## https://github.com/c2nes/javalang/blob/master/javalang/tree.py
#             for path, node in tree.filter(javalang.tree.ClassDeclaration):
#                 #####
#                 ## within each class: 
#                 #####
#                 # print(node.name)
#                 ## fields: 
#                 with open("{}.csv".format(root_PATH), "w") as ff: ## file for field
#                     ff.write("{},{},{},{}\n".format("file", "class", "num_of_field", "num_of_static_field"))
#                 counter_field = 0
#                 counter_field_static = 0
#                 for path1, node1 in node.filter(javalang.tree.FieldDeclaration): ## node1 is each line of FieldDeclaration
#                     if "static" in node1.modifiers:
#                         for variable_declarator in node1.declarators: ## basically one declarator is just one line. Within 1 line, there can be several varaibles. 
#                             # print("static: ", node1.modifiers, variable_declarator.name)
#                             counter_field_static += 1
#                     else:
#                         for variable_declarator in node1.declarators: ## basically one declarator is just one line. Within 1 line, there can be several varaibles. 
#                             # print("not: ", node1.modifiers, variable_declarator.name)
#                             counter_field += 1
#                 # print(file, node.name, counter_field, counter_field_static, "\n")
#                 ## methods:
#                 counter_method = 0
#                 counter_method_static = 0
#                 for path1, node1 in node.filter(javalang.tree.MethodDeclaration): ## node1 is each line of MethodDeclaration
#                     if "static" in node1.modifiers: 
#                         counter_method_static += 1
#                     else:
#                         counter_method += 1
#                 # print(file, node.name, counter_method, counter_method_static, "\n")
#             break